# Trabajo práctico 2 - Simulación

In [6]:
import sys
!{sys.executable} -m pip install scipy

import matplotlib.pyplot as plt
from collections import Counter
from random import random
import math
import scipy.stats as st
import numpy as np

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Ejercicio 1

Utilizando Matlab, Octave o Python (sin utilizar Simpy) se debe determinar con cuál de las siguientes 2 arquitecturas nos permite tener un menor tiempo de procesamiento.
A un procesador arriban instrucciones siguiendo una distribución exponencial con una tasa de 250 instrucciones por
microsegundo.
Las instrucciones se pueden agrupar en las siguientes categorías, y en función de ella es el tiempo que demoran en ser ejecutadas el cual sigue una distribución exponencial con parámetro lambda especificado a continuación.

| Tipo de instrucción | Probabilidad | Tasa del tiempo de ejecución (instrucciones por µseg) |
|:-------------------:|:------------:|:-----------------------------------------------------:|
|        Simple       |      0,6     |                           60                          |
|       Compleja      |      0,4     |                           10                          |

Se sabe que el 65% de las instrucciones requieren buscar datos en memoria para antes de ser ejecutadas.
En este punto se plantean dos alternativas: 


<table>
 <tr>
    <td style="text-align: left; width: 100px;"><b>Alternativa 1:</b></td>
    <td style="text-align: left;">Cada vez que se requiera un dato de memoria se lo busca en memoria principal. Tarea cuyo tiempo corresponde a una distribución exponencial con una tasa de 2000 instrucciones por microsegundo. </td>
 </tr>
 <tr>
    <td style="text-align: left; width: 100px;"><b>Alternativa 2:</b></td>
    <td style="text-align: left;">Implementar un Caché, con el cual existiría una probabilidad de .6 de encontrar el dato requerido y no tener la necesidad de buscarlo en memoria principal.
 El tiempo de acceso a este caché se puede modelar siguiendo una distribución exponencial con una
tasa de 500 instrucciones por microsegundo.
 Cuando el dato no es encontrado en el caché y debe buscarse en memoria principal donde su demora
también sigue una distribución exponencial con una tasa de 1500 instrucciones por microsegundo. </td>
 </tr>
</table>

El arribo de instrucciones a un procesador se pueden modelar como un proceso de Poisson debido a que los arribos siguen una distribución exponencial de 250 instrucciones/us.

Sean:

$\Pi$ = "Arribo de instrucciones a un procesador" ~ Poi($\lambda$ = 250 ins/$\mu$s)

$\Pi_{1}$ = "Arribo de una instrucción simples a un procesador" ~ Poi($\lambda_{1}$ = 0,6 * 250 ins/$\mu$s)

$\Pi_{2}$ = "Arribo de una instruccións complejas a un procesador" ~ Poi($\lambda_{2}$ = 0,4 * 250 ins/$\mu$s)

$T_{P1}$ = "Tiempo que demora una instrucción simple en ser ejecutada" ~ exp($\lambda_{P1}$ = 60 ins/$\mu$s)

$T_{P2}$ = "Tiempo que demora una instrucción compleja en ser ejecutada" ~ exp($\lambda_{P2}$ = 10 ins/$\mu$s)

$P_{M}$ = "Probabilidad de tener que buscar el dato en memoria" = 0.65

<b>Para alternativa 1</b>

$T_{1}$ = "Tiempo que demora el procesador en buscar un dato en memoria" ~ exp($\lambda_{T1}$ = 2000 ins/$\mu$s)

<b>Para alternativa 2</b>

$P_{C}$ = "Probabilidad de que el dato se encuentre en caché" = 0.6

$T_{C}$ = "Tiempo que demora el procesador en buscar el dato en caché" ~ exp($\lambda_{C}$ = 500 ins/$\mu$s)

$T_{NC}$ = "Tiempo que demora el procesador en buscar el dato en memoria y volcarlo en caché" ~ exp($\lambda_{NC}$ = 1500 ins/$\mu$s)

Para resolver el problema, se simulará una variable aleatoria uniforme para identificar si la instrucción es simple o compleja, en base a eso se generará el tiempo que demora en arribar y procesarse. Luego, se simulará otra variable aleatoria uniforme para ver si la instrucción busca datos en memoria o no. Si necesita ir a buscar datos a memoria, entonces se elegirá entre la alternativa 1 o la 2.
Cada experiencia si simulará múltiples veces para corroborar cual es la mejora alternativa en varias experiencias

In [13]:
def inversa_exponencial(u, lam = 1):
    return (-1/lam) * math.log(1 - u)

def tiempo_de_procesamiento(PS, PM, lam1, lam2, lamP1, lamP2, lamT1, PC, lamC, lamNC, alternativa = 1):
    u_P = random() # probabilidad de que la instruccion sea simple
    if u_P < PS: # la instruccion es simple
        Talt1 = inversa_exponencial(random(), lam1) # tiempo de arribo
        Talt1 = Talt1 + inversa_exponencial(random(), lamP1) # tiempo en ser ejecutadas
    else:
        Talt1 = inversa_exponencial(random(), lam2) # tiempo de arribo
        Talt1 = Talt1 + inversa_exponencial(random(), lamP2) # tiempo en ser ejecutadas

    u_M = random() # probabilidad de que tenga que ir a buscar datos a memoria
    if u_M < PM: # busca datos en memoria
        if alternativa == 1:
            Talt1 = Talt1 + inversa_exponencial(random(), lamT1) # tiempo de buscar en memoria con la alternativa 1
        else: # alternativa 2
            u_PC = random()
            if u_PC < PC: # el dato se encuentra en cache
                Talt1 = Talt1 + inversa_exponencial(random(), lamC)
            else:
                Talt1 = Talt1 + inversa_exponencial(random(), lamNC)
                
    return Talt1
        
lam = 250 # tasa de arribo de insutrucciones a un micro

PS = 0.6 # probabilidad que la instruccione sea simple

lam1 = PS * lam  # tasa de arribo de una instruccion simple a un micro
lam2 = (1-PS) * lam # tasa de arribo de una instruccion compuesta a un micro

lamP1 = 60 # tasa que demora una instruccion simple en ser ejecutada
lamP2 = 10 # tasa que demora una instruccion compuesta en ser ejecutada

lamT1 = 2000 # tasa de buscar datos en memoria con la alternativa 1

PM = 0.65 # probabilidad de tener que buscar un dato en memoria

PC = 0.6
lamC = 500
lamNC = 1500


n = 100
for j in range(4):
    t1 = 0
    t2 = 0
    for datos in range(n):
        # alternativa 1
        t1 = t1 + tiempo_de_procesamiento(PS, PM, lam1, lam2, lamP1, lamP2, lamT1, PC, lamC, lamNC, 1)

        # alternativa 2
        t2 = t2 + tiempo_de_procesamiento(PS, PM, lam1, lam2, lamP1, lamP2, lamT1, PC, lamC, lamNC, 2)
    
    print("Para ",n," datos | ","ALTERNATIVA 1 = ",t1," microseg",", ALTERNATIVA 2 = ",t2," microseg")
    n = n * 10

Para  100  datos |  ALTERNATIVA 1 =  5.85061568219554  microseg , ALTERNATIVA 2 =  7.319538418295529  microseg
Para  1000  datos |  ALTERNATIVA 1 =  58.7243006597534  microseg , ALTERNATIVA 2 =  60.862569570092894  microseg
Para  10000  datos |  ALTERNATIVA 1 =  590.8384967059254  microseg , ALTERNATIVA 2 =  593.1942654921464  microseg
Para  100000  datos |  ALTERNATIVA 1 =  5806.544350400481  microseg , ALTERNATIVA 2 =  5884.448548497947  microseg


En la ejecución, observamos que la alternativa 1 parece ser levemente mejor que la 2. Sin embargo, no hay tanto diferencia, por lo que podemos repetir estas experiencias (por ej. para n = 1000) varias veces y contar en cuantas oportunidades t2 > t1

In [20]:
n = 1000
count1 = count2 = 0
for count in range(10000):
    t1 = 0
    t2 = 0
    
    for datos in range(n):
        # alternativa 1
        t1 = t1 + tiempo_de_procesamiento(PS, PM, lam1, lam2, lamP1, lamP2, lamT1, PC, lamC, lamNC, 1)

        # alternativa 2
        t2 = t2 + tiempo_de_procesamiento(PS, PM, lam1, lam2, lamP1, lamP2, lamT1, PC, lamC, lamNC, 2)
    
    if t1 < t2:
        count1 = count1 + 1
    else:
        count2 = count2 + 1
        
if count2 > count1:
    print("La alternativa 2 demoró menos tiempo una mayor cantidad de veces que la alternativa 1")
else:
    print("La alternativa 1 demoró menos tiempo una mayor cantidad de veces que la alternativa 2")

La alternativa 1 demoró menos tiempo una mayor cantidad de veces que la alternativa 2


Entonces efectivamente, cuando realizamos las las experncias y las contamos, en general la alternativa 1 demora menos